
# Evaluation with Mosaic AI Agent Evaluation

In this lab, you will have the opportunity to evaluate a RAG chain model **using Mosaic AI Agent Evaluation Framework.**

**Lab Outline:**

*In this lab, you will complete the following tasks:*

- **Task 1**: Define a custom Gen AI evaluation metric.

- **Task 2**: Conduct an evaluation test using the Agent Evaluation Framework.

- **Task 3**: Analyze the evaluation results through the user interface.

In [0]:
%pip install -U -qq databricks-agents databricks-sdk databricks-vectorsearch langchain-databricks langchain==0.3.7 mlflow tiktoken langchain-community==0.3.7
dbutils.library.restartPython()

In [0]:
%run ../Includes/Classroom-Setup-04


The examples and models presented in this course are intended solely for demonstration and educational purposes.
 Please note that the models and prompt examples may sometimes contain offensive, inaccurate, biased, or harmful content.


In [0]:
print(f"Username:          {DA.username}")
print(f"Catalog Name:      {DA.catalog_name}")
print(f"Schema Name:       {DA.schema_name}")
print(f"Working Directory: {DA.paths.working_dir}")
print(f"Dataset Location:  {DA.paths.datasets}")

Username:          labuser11003544_1753435669@vocareum.com
Catalog Name:      dbacademy
Schema Name:       labuser11003544_1753435669
Working Directory: /Volumes/dbacademy/ops/labuser11003544_1753435669@vocareum_com
Dataset Location:  NestedNamespace (news='/Volumes/dbacademy_news/v01', arxiv='/Volumes/dbacademy_arxiv/v01')


## Lab Overview




## Evaluation Dataset

In this lab, you will work with the same dataset utilized in the demos. This dataset contains sample queries along with their corresponding expected responses, which are generated using synthetic data.

In [0]:
display(DA.eval_df)

request,expected_response,evolution_type,episode_done
"What are the limitations of symbolic planning in task and motion planning, and how can leveraging large language models help overcome these limitations?","Symbolic planning in task and motion planning can be limited by the need for explicit primitives and constraints. Leveraging large language models can help overcome these limitations by enabling the robot to use language models for planning and execution, and by providing a way to extract and leverage knowledge from large language models to solve temporally extended tasks.",simple,true
"What are some techniques used to fine-tune transformer models for personalized code generation, and how effective are they in improving prediction accuracy and preventing runtime errors?","The techniques used to fine-tune transformer models for personalized code generation include ﬁne-tuning transformer models, adopting a novel approach called Target Similarity Tuning (TST) to retrieve a small set of examples from a training bank, and utilizing these examples to train a pretrained language model. The effectiveness of these techniques is shown in the improvement in prediction accuracy and the prevention of runtime errors.",simple,true
How does the PPO-ptx model mitigate performance regressions in the few-shot setting?,"The PPO-ptx model mitigates performance regressions in the few-shot setting by incorporating pre-training and fine-tuning on the downstream task. This approach allows the model to learn generalizable features and adapt to new tasks more effectively, leading to improved few-shot performance.",simple,true
How can complex questions be decomposed using successive prompting?,"Successive prompting is a method for decomposing complex questions into simpler sub-questions, allowing language models to answer them more accurately. This approach was proposed by Dheeru Dua, Shivanshu Gupta, Sameer Singh, and Matt Gardner in their paper 'Successive Prompting for Decomposing Complex Questions', presented at EMNLP 2022.",simple,true
"Which entity type in Named Entity Recognition is likely to be involved in information extraction, question answering, semantic parsing, and machine translation?",Organization,reasoning,true
What is the purpose of ROUGE (Recall-Oriented Understudy for Gisting Evaluation) in automatic evaluation methods?,"ROUGE (Recall-Oriented Understudy for Gisting Evaluation) is used in automatic evaluation methods to evaluate the quality of machine translation. It calculates N-gram co-occurrence statistics, which are used to assess the similarity between the candidate text and the reference text. ROUGE is based on recall, whereas BLEU is based on accuracy.",simple,true
"What are the challenges associated with Foundation SSL in CV, and how do they relate to the lack of theoretical foundation, semantic understanding, and explicable exploration?","The challenges associated with Foundation SSL in CV include the lack of a profound theory to support all kinds of tentative experiments, and further exploration has no handbook. The pretrained LM may not learn the meaning of the language, relying on corpus learning instead. The models cannot reach a better level of stability and match different downstream tasks, and the primary method is to increase data, improve computation power, and design training procedures to achieve better results. The lack of theoretical foundation, semantic understanding, and explicable exploration are the main challenges in Foundation SSL in CV.",simple,true
How does ChatGPT handle factual input compared to GPT-3.5?,"ChatGPT handles factual input better than GPT-3.5, with a 21.9% increase in accuracy when the premise entails the hypothesis. This is possibly related to the preference for human feedback in ChatGPT's RLHF design during model training.",simple,true
What are some of the challenges in understanding natural language commands for robotic navigation and mobile manipulation?,"Some challenges in underst

## Load the Model

A RAG chain has been created and registered for use in this lab. The model details are provided below.

**📌 Note:** If you are using your own workspace to run this lab, you must manually execute **`00 - Build Model / 00-Build Model`**.

In [0]:
import mlflow

catalog_name = "genai_shared_catalog_03"
# schema_name = f"ws_{spark.conf.get('spark.databricks.clusterUsageTags.clusterOwnerOrgId')}"

mlflow.set_registry_uri("databricks-uc")

# model_uri = f"models:/{catalog_name}.{schema_name}.rag_app/1"
model_name = "genai_shared_catalog_03.ws_1286554779316081.rag_app"

## Task 1 - Define A Custom Metric

For this task, define a custom metric to evaluate whether the generated **"ANSWER"** from the RAG chain is easily readable by a non-expert user.

In [0]:
from mlflow.metrics.genai import make_genai_metric_from_prompt

## Prompt for LLM as judge to determine if the generated response is easily readable by non-academic or expert readers
eval_prompt = "Your task is to determine whether the generated response easily readable by non-academic or expert readers. This was the content: '{retrieved_context}'"

## Use Llama-3 as LLM
llm="endpoints:/databricks-meta-llama-3-3-70b-instruct"

## Define the metric
is_readable = make_genai_metric_from_prompt(
    name="is_readable",
    judge_prompt=eval_prompt,
    model=llm,
    metric_metadata={"assessment_type": "ANSWER"},
)

##Task 2 - Run Evaluation Test

Next, run an evaluation using the custom metric you defined. Ensure that you select **Mosaic AI Agent Evaluation** as the evaluation type.


In [0]:
model_uri = "models:/genai_shared_catalog_03.ws_1286554779316081.rag_app/1" 
with mlflow.start_run(run_name="lab_04_agent_evaluation"):
    eval_results = mlflow.evaluate(
        data = DA.eval_df,
        model = model_uri,
        model_type = "databricks-agent",
        extra_metrics=[is_readable]
    )

2025/07/25 11:32:38 WARNING mlflow.pyfunc: The version of Python that the model was saved in, `Python 3.11.11`, differs from the version of Python that is currently running, `Python 3.10.12`, and may be incompatible


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File <command-3791189952706433>, line 3
      1 model_uri = "models:/genai_shared_catalog_03.ws_1286554779316081.rag_app/1" 
      2 with mlflow.start_run(run_name="lab_04_agent_evaluation"):
----> 3     eval_results = mlflow.evaluate(
      4         data = DA.eval_df,
      5         model = model_uri,
      6         model_type = "databricks-agent",
      7         extra_metrics=[is_readable]
      8     )

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-9334fcee-142e-4161-95b7-0efa824da3d4/lib/python3.10/site-packages/mlflow/models/evaluation/base.py:1685, in evaluate(model, data, model_type, targets, predictions, dataset_path, feature_names, evaluators, evaluator_config, custom_metrics, extra_metrics, custom_artifacts, validation_thresholds, baseline_model, env_manager, model_config, baseline_config, inference_params)
   1683      

In [0]:
model = mlflow.pyfunc.load_model(model_uri)
model.predict({"inputs": "What is the capital of France?"})  # Adjust input format as required


## Task 3 - Review Evaluation Results

Review the evaluation results in the **Experiments** section. Examine the following information regarding this evaluation:

- Token usage

- Model metrics

- Results of the custom metric defined earlier ("readability")